In [17]:
import solana
from solana.publickey import PublicKey
from solana.rpc.api import Client
from solana.keypair import Keypair
from solana import system_program
from solana import transaction
from solana.system_program import SYS_PROGRAM_ID, CreateAccountParams, create_account
from solana.transaction import *
from spl.token.constants import TOKEN_PROGRAM_ID, ASSOCIATED_TOKEN_PROGRAM_ID
from spl.token.instructions import initialize_mint, InitializeMintParams
from spl.token._layouts import MINT_LAYOUT
from spl.token.client import Token
from spl.token import instructions
import base64
import base58
from borsh_construct import *
from solana.rpc.commitment import Commitment
import time
import sys
sys.path.append('../../GlobalRepo/')
from global_repo import *

client = Client("https://api.devnet.solana.com")

client.is_connected()

True

In [18]:
program_id = TABLES_PROGRAM_ID
metadata_program_pubkey = METAPLEX_PROGRAM_ID


In [19]:
# client.request_airdrop(PublicKey("DxkiJXK31VEd4R6F44mQRszoUQugpPYMwL9ybcrnt22r"), 2000000000)

In [5]:
accountSchema = CStruct(
    "vault_total" / F32,
    "counter" / U32,
)

request_units = CStruct(
    "units" / U32,
    "additional_fee" / U32   
)

def getAccountInfo(client: Client, account_pk: PublicKey):
    """Fetch account information from RPC, parse out the data and deserialize"""
    comm = Commitment("confirmed")
    res = client.get_account_info(account_pk, comm, encoding='base64')
    data = res['result']
    if isinstance(data, dict):
        return accountSchema.parse(base64.urlsafe_b64decode(data['value']['data'][0]))
    else:
        raise AttributeError(f'Unknown RPC result {data}')

In [6]:
seed = bytes([125,215,212,54,213,98,45,123]*4)
payer_keypair = Keypair.from_secret_key(base58.b58decode("mH4PkhAttpFXSrntbpiJgC8J6wHYFDK5PFWcPN53hMJKziWQucux7MFiW7uXnqKsAY3XBnxUtyVKnomZnD9JxHW"))
client.request_airdrop(payer_keypair.public_key, 2*10**9)
base58.b58encode(payer_keypair.secret_key)

b'mH4PkhAttpFXSrntbpiJgC8J6wHYFDK5PFWcPN53hMJKziWQucux7MFiW7uXnqKsAY3XBnxUtyVKnomZnD9JxHW'

In [7]:
dsol_mint = DSOL_MINT_ID
governor_mint = PublicKey("J3ACcSXB2ACbX2CL4FWxvZh81KRQt4AtF3XhyvpxxrJK")
payer_bump = 5

payer_governor_token_address = instructions.get_associated_token_address(payer_keypair.public_key, governor_mint)

table_mint_keypair = Keypair.generate()

payer_table_associated_token_address = instructions.get_associated_token_address(payer_keypair.public_key, table_mint_keypair.public_key)

(payer_table_data_pda, payer_table_data_pda_bump) = PublicKey.find_program_address([b"table_data_pda", bytes(payer_keypair.public_key), payer_bump.to_bytes(4,"big")], program_id);

(payer_table_metadata_pda, payer_table_metadata_pda_bump) = PublicKey.find_program_address([b"metadata", bytes(metadata_program_pubkey), bytes(table_mint_keypair.public_key)], metadata_program_pubkey) 

(table_sales_pda, table_sales_nonce) = PublicKey.find_program_address([b'table_sales_pda'], program_id)

payer_dsol_token_account = instructions.get_associated_token_address(payer_keypair.public_key, dsol_mint)

(table_mint_authority, nonce) = PublicKey.find_program_address([b"table_mint_authority_pda"], program_id)
payer_table_data_pda

9tJss21NKCaFSU9CGE8wrPC8dH9mQpoJbqNpinqf8f3P

In [8]:
seed = bytes([25,215,212,154,213,198,45,123]*4)
vault_keypair = Keypair.from_seed(seed)
# vault_keypair = Keypair.generate()
# client.request_airdrop(vault_keypair.public_key, 2*10**9)
# time.sleep(5)
vault_dsol_token_account = instructions.get_associated_token_address(vault_keypair.public_key, dsol_mint)

In [9]:

payer_account_meta = AccountMeta(payer_keypair.public_key, True, True)
payer_dsol_token_account_meta = AccountMeta(payer_dsol_token_account, False, True)
payer_governor_token_account_meta = AccountMeta(payer_governor_token_address, False, True)
# payer_table_associated_account_meta = AccountMeta(payer_table_associated_token_address, False, True)
payer_table_data_pda_meta = AccountMeta(payer_table_data_pda, False, True)
payer_table_metadata_meta = AccountMeta(payer_table_metadata_pda, False, True)
vault_account_meta = AccountMeta(vault_keypair.public_key, False, True)
vault_dsol_token_account_meta = AccountMeta(vault_dsol_token_account, False, True)
table_mint_account_meta = AccountMeta(table_mint_keypair.public_key, True, True)
table_mint_authority_meta = AccountMeta(table_mint_authority, False, False)
table_sales_pda_meta = AccountMeta(table_sales_pda, False, True)

rent_account_meta = AccountMeta(solana.sysvar.SYSVAR_RENT_PUBKEY, False, False)
spl_program_meta  = AccountMeta(TOKEN_PROGRAM_ID, False, False)
sys_program_meta = AccountMeta(SYS_PROGRAM_ID, False, False)
sysvar_clock_meta = AccountMeta(solana.sysvar.SYSVAR_CLOCK_PUBKEY, False, False)
associated_program_meta = AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False)
metadata_meta = AccountMeta(metadata_program_pubkey, False, False)
dsol_mint_meta = AccountMeta(dsol_mint, False, False)
compute_program_meta = AccountMeta(PublicKey("ComputeBudget111111111111111111111111111111"), False, False)
gov_account_meta = AccountMeta(governor_mint, False, False)


In [10]:
payer_table_data_pda_meta

AccountMeta(pubkey=9tJss21NKCaFSU9CGE8wrPC8dH9mQpoJbqNpinqf8f3P, is_signer=False, is_writable=True)

In [16]:


# print(dsol_mint)
# print(vault_keypair.public_key)
# print(vault_dsol_token_account)
# print(payer_keypair.public_key)
# print(payer_dsol_token_account)
# print(payer_governor_token_address)
# print(sys_program_meta)
# print(spl_program_meta)
# print(payer_table_metadata_pda)


InstructionEnum = Enum(
    "InitTable" / CStruct("governor_reward" / U32, "payer_bump" / U32),
    "SignTableMint",
    "CreateTableSalesAccount",
    "BurnNFTs" / CStruct("rarity" / U8),
    "MintTable",
    "LockTable",
    "UnlockTable",
    enum_name = "InstructionEnum"
)
InstructionEnum.build(InstructionEnum.enum.LockTable())

b'\x00\xf4\x01\x00\x00\x05\x00\x00\x00'

In [24]:
# payer_bump = 0
# (table_data_pda, table_data_pda_bump) = PublicKey.find_program_address([b"table_data_pda", bytes(payer_account_meta.pubkey), payer_bump.to_bytes(4,"little")], program_id);
# payer_table_data_pda_meta = AccountMeta(payer_table_data_pda, False, True)

vault_pda, _ = PublicKey.find_program_address([b"Dsol_vault_tables"], program_id)
vault_pda_dsol_token_account_meta = AccountMeta(instructions.get_associated_token_address(vault_pda, dsol_mint), False, True)

(pending_pda, pending_nonce) = PublicKey.find_program_address([b'pending_tables'], program_id)
pending_tables_meta = AccountMeta(pending_pda, False, True)

main_vault_assoc = instructions.get_associated_token_address(VAULT, dsol_mint)

main_vault_meta = AccountMeta(main_vault_assoc, False, True)

gov_data_pubkey, _ = PublicKey.find_program_address([b'governor_data_pda', bytes(governor_mint)], GOVERNOR_PROGRAM_ID)
gove_data_meta = AccountMeta(gov_data_pubkey, False, True)


In [25]:
def init_table_mint():
    accounts = [
        payer_account_meta,
        payer_dsol_token_account_meta,
        payer_governor_token_account_meta,
        # payer_table_associated_account_meta,
        payer_table_data_pda_meta,
        # payer_table_metadata_meta,
        # vault_account_meta,
        vault_pda_dsol_token_account_meta,
        # table_mint_account_meta,
        # table_mint_authority_meta,
        table_sales_pda_meta,
        # sys_program_meta,
        spl_program_meta,
        # rent_account_meta,
        sysvar_clock_meta,
        pending_tables_meta,
        main_vault_meta,
        gove_data_meta,
        gov_account_meta,

        # compute_program_meta,
        spl_program_meta,
        spl_program_meta,
        sys_program_meta,
        # associated_program_meta,
        # spl_program_meta,
        # metadata_meta,
        # metadata_meta,
        # spl_program_meta,
        # spl_program_meta,
        # sys_program_meta,
    ]


    transaction = Transaction()

    # request_units = CStruct(
    # "units" / U32,
    # "additional_fee" / U32   
    # )


    # transaction.add(
    #     TransactionInstruction(
    #         [],
    #         compute_program_meta.pubkey,
    #         request_units.build({"units": 500000, "additional_fee": int(0.01*10**9)})
    #     )
    # )

    transaction.add(
        TransactionInstruction(
            accounts,
            program_id,
            InstructionEnum.build(InstructionEnum.enum.InitTable(governor_reward = 500, payer_bump = payer_bump))
        )
    )
    
    return f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair)['result']}?cluster=devnet"

init_table_mint()

# time.sleep(5)

RPCException: {'code': -32002, 'message': 'Transaction simulation failed: Error processing Instruction 0: custom program error: 0xb', 'data': {'accounts': None, 'err': {'InstructionError': [0, {'Custom': 11}]}, 'logs': ['Program 7fmmvCJ1uH4sfLCdZrDAjJnfHejwtFbYL7aG8nnNR6Yz invoke [1]', 'Program 7fmmvCJ1uH4sfLCdZrDAjJnfHejwtFbYL7aG8nnNR6Yz consumed 8711 of 200000 compute units', 'Program 7fmmvCJ1uH4sfLCdZrDAjJnfHejwtFbYL7aG8nnNR6Yz failed: custom program error: 0xb'], 'unitsConsumed': 0}}

In [ ]:
program_id

7fmmvCJ1uH4sfLCdZrDAjJnfHejwtFbYL7aG8nnNR6Yz

In [ ]:
# def mint_avatar():
#     accounts = [
#         # program_id_account_meta,
#         payer_account_meta,
#         payer_dsol_token_account_meta,
#         vault_account_meta, 
#         vault_dsol_token_account_meta,
#         mint_account_meta, 
#         rent_account_meta, 
#         associated_account_meta,
#         spl_program_meta,
#         AccountMeta(SYS_PROGRAM_ID, False, False),
#         mint_authority_meta,
#         avatar_data_pda_meta,
#         sales_pda_meta,
#         AccountMeta(metadata_pda, False, True),
#         sysvar_clock_meta,

#         spl_program_meta,
#         AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False),
#         spl_program_meta,
#         spl_program_meta,
#         AccountMeta(metadata_program_pubkey, False, False),
#     ]   

#     transaction = Transaction()
#     transaction.add(TransactionInstruction(
#         accounts,
#         program_id,
#         bytes([0])
#     ))
    
#     return f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair, mint_account, vault_keypair)['result']}?cluster=devnet"

# mint_avatar()

In [ ]:

# accountSchema = CStruct(
#     "vault_total" / F32,
#     "counter" / U32,
# )

# (sales_pda, sales_nonce) = PublicKey.find_program_address([b'sales_pda', bytes(program_id)], program_id)
# print(f"PDA: {sales_pda, sales_nonce}")

# getAccountInfo(client,sales_pda)

In [24]:
accountSchema = CStruct(
    "date_created" / U32,
    "id" / U32,
    "rarity" / U8,
    "creators" / Vec(U8[32]),
    "num_creators" / U8,
    "governor_reward" / U32,
    "minted" / Bool,
    "is_on_lobby" / Bool,
)

table_data_pda, _ = PublicKey.find_program_address([b"table_data_pda", bytes(PublicKey("FXyQAAPKHqxGcKtF9jPnchV1aL2iUdEfBEkkKpJhr6i1"))], TABLES_PROGRAM_ID)

getAccountInfo(client, table_data_pda)

Container(date_created=1646664889, id=3, rarity=1, creators=ListContainer([ListContainer([55, 150, 108, 206, 0, 165, 248, 105, 20, 106, 122, 114, 14, 182, 186, 150, 210, 119, 119, 108, 0, 117, 0, 164, 110, 197, 167, 182, 212, 40, 30, 60]), ListContainer([181, 81, 167, 153, 25, 48, 166, 0, 192, 140, 146, 52, 85, 156, 4, 134, 93, 134, 45, 240, 124, 33, 189, 35, 112, 53, 252, 186, 125, 103, 15, 78]), ListContainer([248, 3, 43, 108, 102, 68, 126, 82, 103, 220, 76, 148, 58, 80, 17, 225, 55, 55, 238, 207, 198, 123, 126, 109, 181, 166, 230, 5, 139, 83, 140, 253]), ListContainer([176, 93, 13, 189, 195, 250, 118, 232, 31, 170, 192, 162, 165, 238, 168, 72, 44, 42, 245, 93, 15, 153, 95, 161, 128, 156, 99, 137, 162, 52, 115, 173]), ListContainer([136, 188, 64, 60, 113, 132, 31, 30, 77, 154, 45, 218, 43, 245, 8, 103, 243, 100, 6, 245, 175, 164, 250, 227, 94, 106, 7, 13, 162, 244, 77, 48])]), num_creators=5, governor_reward=500, minted=True, is_on_lobby=False)

In [ ]:
# time.sleep(120)

# new_accounts = [
#     mint_account_meta,
#     associated_account_meta,
#     spl_program_meta,
#     mint_authority_meta,
#     avatar_data_pda_meta,
#     sysvar_clock_meta,

#     spl_program_meta
# ]

# transaction = Transaction()
# transaction.add(TransactionInstruction(
#     new_accounts,
#     program_id,
#     bytes([1])
# ))
# f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair, mint_account)['result']}?cluster=devnet"